In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry import Point, LineString
import geopandas as gpd
from matplotlib.ticker import AutoMinorLocator

In [2]:
url_template = 'https://www.ndbc.noaa.gov/view_text_file.php?filename=46042h{year}.txt.gz&dir=data/historical/stdmet/'
    
# create an empty list to store the data frames
dfs = []

# loop through the years
for year in range(2000, 2005):

    # construct the file path
    url = url_template.format(year=year)
    
    # parse the data and create a data frame
    df = pd.read_csv(url, delim_whitespace=True, skiprows=1, header=None, names=['year', 'month', 'day', 'hour', 'wdir', 'wspd', 'gst', 'wvht', 'dpd', 'apd', 'mwd', 'pres', 'atmp', 'wtmp', 'dewp', 'vis', 'ptdy', 'tide'])
    
    # add the data frame to the list
    dfs.append(df)

# concatenate all the data frames into a single data frame
df = pd.concat(dfs, ignore_index=True)

# print the resulting data frame
df.head()

,year,month,day,hour,wdir,wspd,gst,wvht,dpd,apd,mwd,pres,atmp,wtmp,dewp,vis,ptdy,tide
0,2000,1,1,0,297,4.9,6.1,1.22,7.14,5.68,323,1017.4,11.0,11.6,6.5,99.0,NaN,NaN
1,2000,1,1,1,305,5.2,6.4,1.33,6.67,5.60,320,1017.6,11.1,11.7,6.7,99.0,NaN,NaN
2,2000,1,1,2,302,5.5,6.7,1.29,7.14,5.41,324,1018.1,11.2,11.6,6.7,99.0,NaN,NaN
3,2000,1,1,3,304,2.4,3.1,1.22,7.14,5.70,320,1018.1,11.3,11.6,6.6,99.0,NaN,NaN
4,2000,1,1,4,292,4.7,6.1,1.27,6.67,5.90,321,1018.5,11.5,11.6,7.2,99.0,NaN,NaN


In [ ]:
dfs2 = []
for year in range(2005, 2023):
    url = url_template.format(year=year)
    df2 = pd.read_csv(url, delim_whitespace=True, skiprows=1, header=None, names=['year', 'month', 'day', 'hour', 'minute', 'wdir', 'wspd', 'gst', 'wvht', 'dpd', 'apd', 'mwd', 'pres', 'atmp', 'wtmp', 'dewp', 'vis', 'ptdy', 'tide'])
    dfs2.append(df2)

df2 = pd.concat(dfs2, ignore_index=True)

# print the 2000 - 2005 range
df2.head()

In [ ]:
data_list = [df, df2]
data_list = pd.concat(data_list, ignore_index=True)
print(data_list.head())

In [ ]:
data_list

In [ ]:
# need to get rid of strings 
data_clean = data_list[data_list['wvht'] != 'm']
# data_clean

# to datetime index 
date_cols = ['year','month','day','hour']
data_clean['time'] = pd.to_datetime(data_clean[date_cols])
data_clean.set_index('time',inplace=True)
data_clean.drop(date_cols,axis=1,inplace=True)
cols_to_drop = ['pres','atmp','dewp','vis','ptdy','tide','wspd','gst']
# drop extra columns and drop bad sample values (99.0 or 999.0)
data_clean.drop(cols_to_drop,axis=1,inplace=True)
data_clean['wvht'] = data_clean['wvht'].astype(float)
# more
data_clean.wdir = data_clean['wdir'].astype(float)
data_clean.mwd = data_clean['mwd'].astype(float)

print(data_clean)

In [ ]:
data_clean.wvht.mean()
# data_clean.wdir.max()

In [ ]:
data_clean.wdir = data_clean['wdir'].astype(float)
data_clean.wdir.mean()
# data_clean.wdir.max()

In [ ]:
plt.plot(data_clean.wvht)

In [ ]:
#remove bad wvht values === 99.0
data_wave = data_clean.loc[data_clean["wvht"] < 98]
print(data_wave.shape)
# remove wdir where values > 999.0
data = data_clean.loc[(data_clean["wdir"] < 999.0) & (data_clean["wvht"] < 98) ]
print(data.wdir.max())
print(data_wave)

In [ ]:
plt.plot(data_wave.wvht)

In [ ]:
### Convert wvht: m to ft
data_wave_ft = data_wave.wvht * 3.28084
print(data_wave_ft)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,4))
ax.plot(data_wave.wvht * 3.28084, color='green')
ax.set_title('Wave Height 2000-2023\n')
ax.set_xlim(data_wave.index[0],data_wave.index[-1])
#### shows all wave height measurements in Monterey Bay, California

In [ ]:
avg_by_year = data.groupby(data.index.year).mean()
# avg_wave_height_by_year = data.groupby('year')['wvht'].mean()
# avg_wave_height_by_month = df.groupby('month')['wvht'].mean()
# print(avg_by_year)
avg_by_year_wvht = data_wave_ft.groupby(data_wave_ft.index.year).mean()
# print(avg_by_year_wvht)
# print(avg_wave_height_by_month)

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
# axs[0].plot(avg_by_year.wvht)
axs[0].plot(avg_by_year_wvht)
axs[1].plot(avg_by_year.wdir)
axs[2].plot(avg_by_year.mwd)

In [ ]:
avg_by_month = data.groupby(data.index.month).mean()
# avg_wave_height_by_year = data.groupby('year')['wvht'].mean()
# avg_wave_height_by_month = df.groupby('month')['wvht'].mean()
# print(avg_by_month)
avg_by_month_wvht = data_wave_ft.groupby(data_wave_ft.index.month).mean()
# print(avg_by_month_wvht)
# print(avg_wave_height_by_month)

#### by month
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))

axs[0].plot(avg_by_month_wvht)
axs[0].set_title('Mean Monthly Wave Height 2000-2023')
axs[2].plot(avg_by_month.wdir)
axs[2].set_title('Mean Monthly Wind Direction 2000-2023')
axs[1].plot(avg_by_month.mwd)
axs[1].set_title('Mean Monthly (FFT Mean) Wave Direction 2000-2023')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,4))
ax.plot(avg_by_month_wvht, color='red')
ax.set_title('Mean Monthly Wave Height 2000-2023\n')
#### shows the seasonal variablility in Monterey Bay, ca

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 4))
# axs[0].plot(avg_by_year.wvht)
ax.plot(avg_by_year_wvht,color='blue')
ax.set_title('Mean Yearly Wave Height 2000-2023\n')

### Doesnt answer the question is frequency of large wave events is changing through time?

In [ ]:
# avg by day isn't helpful
years = np.arange(2000,2023,1)
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
plt.scatter(years, avg_by_year_wvht, marker='o',color='blue')
# ax.set_ylim(6,9)
# ax.set_title('Year vs Average Wave Height (ft)')
x= years
y= avg_by_year_wvht
poly3 = np.polyfit(x,y,3)
poly3_object = np.poly1d(poly3)
poly3_val = np.polyval(poly3,x)
plt.plot(x,poly3_val, color='black',linestyle='dashed',label='Third Order Fit')

#### very short-sighted, further account for MJO, ENSO would better predict

In [ ]:
# number of days above 15 ft?
# data_wave_ft & data are df's left using
data

In [ ]:
#### Wave direction and Wave Height are roughly correlated by monthly means
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(avg_by_month_wvht, color="blue")
ax2 = ax.twinx()
ax2.plot(avg_by_month.mwd, color="orange")
ax.set_ylabel('Wave Height [ft]', color='blue')
ax2.set_ylabel('(Mean) Wave Direction [degree]', color='orange')
ax.set_xlabel('Month')

In [ ]:
'''data_wave = data_clean.loc[data_clean["wvht"] < 98]'''
# print(data_wave)
#### resample by daily max and then select for only above 15 ft
data_wave_resample_daily_max = data_wave['wvht'].resample('1D').max()
# print(data_wave_resample_daily_max)
data_wave_resample_daily_max_ft = data_wave_resample_daily_max * 3.28084
data_wave_resample_daily_max_ft_clean = data_wave_resample_daily_max_ft.loc[data_wave_resample_daily_max_ft > 10] 
# print(data_wave_resample_daily_max_ft_clean)

#### convert series back to pd df and it's index is datetime!
new_data = data_wave_resample_daily_max_ft_clean.to_frame()
new_data2 = new_data.copy()

# new_data2.index

# data_wave = data_clean.loc[data_clean["wvht"] < 98]
# plt.scatter(np.arange(0,93),data_wave_resample_daily_max_ft_clean)
# df_new_2.to_csv('data2.txt', sep='\t')
# df['date'] = pd.to_datetime(df['date'])
yearly_counts = new_data2.groupby(new_data2.index.year).count()
print(yearly_counts)
plt.plot(yearly_counts)

In [ ]:
## read in 2023 file to find swell that took out piers?
